In [8]:
!pip install 'markitdown[all]' ffmpeg-downloader
!pip install llama-index-llms-openai_like llama-index-embeddings-openai iprogress ipywidgets jupyter 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [jupyter]m2/5 [ipywidgets]


In [9]:
import os
from markitdown import MarkItDown

result = None
if os.path.exists("./data/SENSIBLE_DATA.xlsx") is False:
    md = MarkItDown(enable_plugins=False) # Set to True to enable plugins
    result = md.convert("./data/SENSIBLE_DATA.xlsx")
else:
    print("SENSIBLE_DATA.xlsx already exists, skipping conversion.")

SENSIBLE_DATA.xlsx already exists, skipping conversion.


In [10]:
if result:
    with open("./data/result.md", "w", encoding="utf-8") as f:
        f.write(result.text_content)
else:
    print("File already exists.")

File already exists.


In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

base_url = os.getenv("IONOS_BASE_URL", "http://localhost:11434")
api_key = os.getenv("IONOS_API_KEY", "your_api_key_here")

os.environ["OPENAI_API_BASE"] = base_url
os.environ["OPENAI_API_KEY"] = api_key

In [12]:
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
}

In [13]:
from llama_index.llms.openai_like import OpenAILike

llama_3_3 = 'meta-llama/Llama-3.3-70B-Instruct'

llm = OpenAILike(
    api_base=base_url,
    temperature=0,
    model=llama_3_3,
    is_chat_model=True,
    default_headers=headers,
    api_key=api_key,
    context_window=128000,  # Adjusted to a more reasonable value for Llama 3.3-70B-Instruct
)

In [14]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

embed_model = OpenAIEmbedding(
  model_name=embed_model_name,
  api_base=base_url,
  api_key=api_key,
  default_headers=headers,
  embed_batch_size=10
)